In [86]:
# Load libraries

from IPython.display import display
from requests_html import HTMLSession
from bs4.builder import XMLParsedAsHTMLWarning
import pandas as pd
import requests as rq
import warnings


In [2]:
# Set library config

warnings.filterwarnings('ignore', category=XMLParsedAsHTMLWarning)
pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 50)
pd.set_option('display.max_rows', 100)

In [29]:
# Load TSV and grab relevant information

metadata = pd.read_csv('../metadata.tsv', sep='\t')
data = metadata[['show_name', 'episode_name',
                'rss_link', 'duration','show_filename_prefix', 'episode_filename_prefix']].copy()

In [78]:
# Isolate shows with enough episodes

at_least_ep = 350

num_of_ep = data['show_name'].value_counts(dropna=False)
lengthy_pods = num_of_ep[num_of_ep > at_least_ep].to_frame()
lp_list = lengthy_pods.index.to_list()
long_shows = data[data['show_name'].isin(lp_list)]
len(lp_list)

10

In [79]:
# Groupby show_name

list_vals = lambda x: list(x)
common_val = lambda x: x.mode()
pods = long_shows.groupby('show_name')[['show_name', 'episode_name', 'rss_link', 'duration', 'show_filename_prefix', 'episode_filename_prefix'
                                        ]].agg({'episode_name': list_vals, 'rss_link': common_val, 'duration': list_vals, 'show_filename_prefix': list_vals, 'episode_filename_prefix': list_vals
                                               }).reset_index()

In [80]:
# Save to CSV

#pods.to_csv("../refined_metadata.csv", index=False)
#test_csv = pd.read_csv('../refined_metadata.csv', sep=',')
#test_csv

In [2]:
# for show in range(len(pods)):
#     rss = pods['rss_link'][show]
#     episodes = pods['episode_name'][show]

In [1]:
# import sys
# !{sys.executable} -m pip install requests_html